# Automatizar ejecución y exportación de simulaciones en omnet

La idea de este notebook es automatizar la ejecución de simulaciones en omnet y la exportación de los resultados a un archivo csv.
que manera de reenegar

## Definir el archivo a cambiar
Acá esta la mayoria de datos que se tienen que cambiar según la maquina de cada uno
- `path_carpeta_trabajo`: define donde tienen el lab 
- `omnet_ejecutable`: el nombre del ejecutable del codigo del omnet
- `path_donde_esta_el_omnet` este voy a hacer un parrafo aparte para explicarlo
- `caso` el caso a testear
- `parameter_values`: los valores de la media de generación a ver
- `caso` el caso a testear
- `exportados` donde guardar los datos

Luego los otros datos importantes se generan solos

### Explicacion del path_donde_esta_el_omnet
Hay una tool que es `scavetool` que exporta los datos. Los tutos onlines y el chatgpt me decian de usarla desde la terminal y no encontraba la forma, por lo que en vez de importarlo al bash o algo como la gente lo que hago es definir el path donde esta el ejecutable y lo llamo desde ahi. Medio gris, lo se, pero funciona.

In [45]:
import os
import re

## Datos especificos de cada uno
path_carpeta_trabajo = "/home/gasbon/Documents/Famaf/actual/redes/labs/lab3"
omnet_ejecutable = "Lab03"

## esto parece SUPER engorroso y al pedo, pero cuando vea una mejor solucion la cambio
path_donde_esta_el_omnet = "/home/gasbon/Downloads/omnetpp-6.0.1"
path_scave = f"{path_donde_esta_el_omnet}/bin/opp_scavetool"

## Datos segun el caso y la media
parameter_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
caso = "caso2"

# exportados = f"{path_carpeta_trabajo}/exportados"
exportados = f"/home/gasbon/Documents/Famaf/actual/redes/labs/lab3/lab3python/exportados/{caso}"
os.makedirs(exportados, exist_ok=True)

if not os.path.exists(path_carpeta_trabajo):
    print("No se encontró la carpeta de trabajo")

omnet_ini = f"{path_carpeta_trabajo}/omnetpp.ini"
omnet_results = f"{path_carpeta_trabajo}/results"



In [46]:


pattern = r'Network\.nodeTx\.gen\.generationInterval\s=\sexponential\(\d+\.\d+\)'

# general_anf = "General.anf"

for i in parameter_values:
    """Corre todas las simulaciones con los valores de i
    El output es mucho asi que se corta, pero es completo posta creanme
    """
    with open(omnet_ini, 'r') as file:
        config_content = file.read()

    modified_config = re.sub(pattern, f'Network.nodeTx.gen.generationInterval = exponential({i})', config_content)

    with open(omnet_ini, 'w') as file:
        file.write(modified_config)

    # esto hace que se corra y las flags es para que sea instantaneo
    os.chdir(path_carpeta_trabajo)
    error = os.system(f"./{omnet_ejecutable} -f {omnet_ini} -u Cmdenv --cmdenv-express-mode=true")
    if error:
        print("Hubo un error")
        break

    # esto es para que se exporten los resultados
    unidad, decimal = str(i).split(".")
    juntado = unidad + decimal
    nombre = f"{exportados}/{caso}_{juntado}"
    os.system(f"{path_scave} export -F CSV-R -o {nombre}.csv  {omnet_results}/*.vec")

OMNeT++ Discrete Event Simulation  (C) 1992-2022 Andras Varga, OpenSim Ltd.
Version: 6.0.1, build: 220831-e4f07d04b8, edition: Academic Public License -- NOT FOR COMMERCIAL USE
See the license for distribution terms and warranty disclaimer

Setting up Cmdenv...

Loading NED files from .:  1

Preparing for running configuration General, run #0...
Assigned runID=General-0-20230512-00:21:36-31412
Setting up network "Network"...
Initializing...

Running simulation...
** Event #0   t=0   Elapsed: 1.1e-05s (0m 00s)  0% completed  (0% total)
     Speed:     ev/sec=0   simsec/sec=0   ev/simsec=0
     Messages:  created: 4   present: 4   in FES: 2
** Event #12931   t=200   Elapsed: 0.00813s (0m 00s)  100% completed  (100% total)
     Speed:     ev/sec=0   simsec/sec=0   ev/simsec=0
     Messages:  created: 1983   present: 214   in FES: 7

<!> Simulation time limit reached -- at t=200s, event #12931

Calling finish() at end of Run #0...

End.
Exported 7 vectors
OMNeT++ Discrete Event Simulation 